In [1]:
from Utils.loadset import getDataSet
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_probability as tfp


tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions
import os
from trainer import Trainer
try:
    from Utils.connection_cfg import *
except Exception as e:
    PSWD = None
    USRN = None
    
from Utils.Data import dataWrapper
from Utils.transform import ToCategorical, cutOut


Using TensorFlow backend.


Num GPUs Available: 1


In [2]:
dimension = (64,64)
batch_size = 100
channels = 5
optimizer = Adam( lr = 1e-5 )
slices = [256,320,256,320]
cutOutFrame = cutOut(slices)

categorical_list = [0,1,5,15,20,25,30,35,40,50,60,80,100,130,160,200]
categorical = ToCategorical(categorical_list)

PRETRAINING_TRANSFORMATIONS = [cutOutFrame]
TRANSFORMATIONS = [categorical]
#TRANSFORMATIONS = None

def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true)


def provideData(flatten=False,dimension=dimension,batch_size=60,transform=None,preTransformation=None):

    getDataSet(DatasetFolder,year=[2017],username=USRN,pswd=PSWD)
    train,test = dataWrapper(PathToData,
                            dimension=dimension,
                            channels=channels,
                            batch_size=batch_size,
                            overwritecsv=True,
                            flatten=flatten,
                            onlyUseYears=[2017],
                            transform=transform,
                            preTransformation=preTransformation)
    
    return train,test
DatasetFolder = "./Data/RAW"
PathToData = os.path.join(DatasetFolder,"MonthPNGData")

train, test = provideData(dimension=dimension,
                          batch_size=batch_size,
                          transform=TRANSFORMATIONS,
                          preTransformation=PRETRAINING_TRANSFORMATIONS)


Found Year :  2017 => won't download this year again... please check for consistency
Finished Loading Dataset
 


In [3]:
def convLayer(inp,nbrLayer,channel,activation="selu"):
    assert nbrLayer > 0, "In Function convLayer nbrLayer > 0 ?"
    layer = Conv2D(channel, kernel_size=(3, 3), padding="same") (inp)
    layer = Activation(activation)(layer)
    layer = BatchNormalization()(layer)
    
    for i in range(1,nbrLayer):
        layer = Conv2D(channel, kernel_size=(3, 3), padding="same")  (layer)
        layer = Activation(activation)(layer)
        layer = BatchNormalization()(layer)
    return layer

#def FullUnetLoop(input_shape,down_channels=[64,128,256,512,1024],downLayer=2,activation="relu"):

def Categorical_fullUnet(input_shape,down_channels=[64,128,256,512],downLayer=2,activation="relu"):


    
    inputs = Input(shape=input_shape)
    
    layer = Conv2D(down_channels[0], kernel_size=(3, 3), padding="same") (inputs)
    layer = Activation(activation)(layer)
    layer = BatchNormalization()(layer)
    
    layer = Conv2D(down_channels[0], kernel_size=(3, 3), padding="same") (layer)
    layer = Activation(activation)(layer)
    firstLayer = BatchNormalization()(layer)
    
    pool  = MaxPooling2D((2, 2), strides=(2, 2))(firstLayer)
    
    layerArray = []
    
    for channel in down_channels[1:]:
        
        layer = convLayer(pool,downLayer,channel)
       
        if channel != down_channels[-1]:
            layerArray.append(layer)
            pool  = MaxPooling2D((2, 2), strides=(2, 2))(layer)
            
    for i,channel in enumerate(reversed(down_channels[:-1])):
        
        layer = Conv2DTranspose(channel,(3, 3),strides=(2,2),padding="same")(layer)
        layer = Activation(activation)(layer)
        layer = BatchNormalization() (layer)
        
        if len(layerArray) >= (i+1):
            layer = concatenate([layerArray[-(i+1)], layer], axis=3)
        else:
            layer = concatenate([firstLayer, layer], axis=3)
        
        layer = convLayer(layer,downLayer,channel)
        
    output = Conv2D(len(categorical_list)-1, kernel_size=(1, 1), padding="same",activation="softmax") (layer)
    #output = Activation("sigmoid")
    #output = Flatten()(output)
    #output = Dense(4096)(output)
    output = tfpl.OneHotCategorical(event_size=(*dimension,len(categorical_list)-1))(output)
    
    model = Model(inputs=inputs, outputs=output)
    return model

In [4]:
t = Trainer(Categorical_fullUnet,
                    NLL,
                    (train,test),
                    batch_size = batch_size,
                    optimizer=optimizer,
                    dimension = dimension,
                    channels = channels,
                    metrics = ["mse","mae"])

[Load file failed]  ./model_data/Categorical_fullUnet_function/Categorical_fullUnet_function64x64x5.h5
[Load file failed]  ./model_data/Categorical_fullUnet_function/Categorical_fullUnet_function64x64x5history.json
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 5)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 64)   2944        input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 64, 64)   0           conv2d[0][0]                     
_____________________________________________________________________________

In [ ]:
t.fit(20)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 788 steps, validate for 262 steps
Epoch 1/20
788/788 [==============================] - 199s 253ms/step - loss: 2.3105 - mse: 0.1189 - mae: 0.1189 - val_loss: 1.9517 - val_mse: 0.1138 - val_mae: 0.1138
Epoch 2/20
788/788 [==============================] - 193s 245ms/step - loss: 2.0892 - mse: 0.1159 - mae: 0.1159 - val_loss: 1.9954 - val_mse: 0.1146 - val_mae: 0.1146
Epoch 3/20
788/788 [==============================] - 193s 245ms/step - loss: 2.0396 - mse: 0.1152 - mae: 0.1152 - val_loss: 1.9910 - val_mse: 0.1145 - val_mae: 0.1145
Epoch 4/20
 30/788 [>.............................] - ETA: 2:31 - loss: 2.0223 - mse: 0.1149 - mae: 0.1149

In [ ]:
model = t.model
x,y = test[0]
print(y.shape)
pred = model(x).mean()

In [ ]:
import numpy as np
class fromCategorical(object):
    def __init__(self, conditions):
        super(fromCategorical, self).__init__()
        self.conditions = np.array(conditions)
        self.numClasses = len(self.conditions) -1
        
    def __call__(self,array):
        img = np.zeros( (array.shape[0],array.shape[1],1) )
        x,y = array.shape[:2]
        
        def mapTo(l):
            if np.argmax(l) != 0:
                img[i,j] = self.conditions[np.argmax(l)]
        for i in range(x):
            for j in range(y):
                mapTo(array[i,j])
        return img


In [ ]:
!pip install matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.pyplot import figure
import numpy as np
%matplotlib inline
history = t.history
def plotHistory():
    # Plot training & validation accuracy values
    plt.plot(history['mse'])
    plt.plot(history['val_mse'])
    plt.title('Model MSE')
    plt.ylabel('MSE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
plotHistory()

In [ ]:
fromcat = fromCategorical(categorical_list)
img = []
for i in range(pred.shape[0]):
    img.append(fromcat(pred[i,:,:,:]))

label = []
for i in range(y.shape[0]):
    label.append(fromcat(y[i,:,:,:]))

In [ ]:
fig, axes = plt.subplots(20,2, figsize=(8,8*20),dpi=64)

for batch,img in enumerate(pred[:20]):
    axes[batch,0].imshow(img[:,:,0],cmap="gray")
    axes[batch,1].imshow(label[batch][:,:,0],cmap="gray")